# Transfer Learning MNIST

* Train a simple convnet on the MNIST dataset the first 5 digits [0..4].
* Freeze convolutional layers and fine-tune dense layers for the classification of digits [5..9].

### Import MNIST data and create 2 datasets with one dataset having digits from 0 to 4 and other from 5 to 9 

#### Import the mnist dataset from keras datasets

In [1]:
import numpy as np
import datetime
import keras

from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils

Using TensorFlow backend.


In [2]:
np.random.seed(1696)

#### Creating two datasets one with digits below 5 and one with 5 and above

In [3]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train_lt5 = x_train[y_train < 5]
y_train_lt5 = y_train[y_train < 5]
x_test_lt5 = x_test[y_test < 5]
y_test_lt5 = y_test[y_test < 5]

x_train_gt5 = x_train[y_train >= 5]
y_train_gt5 = y_train[y_train >= 5] - 5 
x_test_gt5 = x_test[y_test >= 5]        
y_test_gt5 = y_test[y_test >= 5] - 5

### Check 

Verify shapes of x_train, y_train, x_test and y_test for both the datasets with the below given shapes.

In [4]:
print(x_train_lt5.shape)
print(y_train_lt5.shape)
print(x_test_lt5.shape)
print(y_test_lt5.shape)

(30596, 28, 28)
(30596,)
(5139, 28, 28)
(5139,)


In [5]:
print(x_train_gt5.shape)
print(y_train_gt5.shape)
print(x_test_gt5.shape)
print(y_test_gt5.shape)

(29404, 28, 28)
(29404,)
(4861, 28, 28)
(4861,)


### Let us take only the dataset (x_train, y_train, x_test, y_test) for Integers 0 to 4 in MNIST
### Reshape x_train and x_test to a 4 Dimensional array (channel = 1) to pass it into a Conv2D layer

In [6]:
x_train_lt5 = x_train_lt5.reshape(x_train_lt5.shape[0],28,28,1)
x_test_lt5 = x_test_lt5.reshape(x_test_lt5.shape[0],28,28,1)

### Change into float32 datatype and Normalize x_train and x_test by dividing it by 255.0

In [7]:
x_train_lt5 = x_train_lt5.astype('float32')
x_test_lt5 = x_test_lt5.astype('float32')
x_train_lt5 /= 255
x_test_lt5 /= 255

### Check

Verify the shapes of the X_train and X_test with the shapes given below.

In [8]:
print('X_train shape:', x_train_lt5.shape)
print(x_train_lt5.shape[0], 'train samples')
print(x_test_lt5.shape[0], 'test samples')

X_train shape: (30596, 28, 28, 1)
30596 train samples
5139 test samples


In [9]:
print('X_train shape:', x_train_lt5.shape)
print('X_test shape:', x_test_lt5.shape)

X_train shape: (30596, 28, 28, 1)
X_test shape: (5139, 28, 28, 1)


### Use One-hot encoding to divide y_train and y_test into required no of output classes

In [10]:
nb_classes = 5

In [11]:
# convert class vectors to binary class matrices
y_train_lt5 = np_utils.to_categorical(y_train_lt5, nb_classes)
y_test_lt5 = np_utils.to_categorical(y_test_lt5, nb_classes)

### Build a sequential model with 2 Convolutional layers with 32 kernels of size (3,3) followed by a Max pooling layer of size (2,2) followed by a drop out layer to be trained for classification of digits 0-4  

In [12]:
batch_size = 64
nb_epoch = 5
input_shape = (28,28)

# number of convolutional filters
nb_filters = 32
# size of pooling area for max pooling
pool_size = 2
# convolution  size
kernel_size = 3

### Post that flatten the data and add 2 Dense layers with 128 neurons and neurons = output classes with activation = 'relu' and 'softmax' respectively. Add dropout layer inbetween if necessary  

In [13]:
model = Sequential()
# define two groups of layers: feature (convolutions) and classification (dense)
model.add(Convolution2D(nb_filters, kernel_size, kernel_size, input_shape=(28,28,1)))
model.add(Activation('relu'))
model.add(Convolution2D(nb_filters, kernel_size, kernel_size))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(pool_size, pool_size)))
model.add(Dropout(0.25))
model.add(Flatten())

##classification_layers
model.add(Dense(128, activation ='relu'))
model.add(Dropout(0.7))
model.add(Dense(nb_classes, activation = 'softmax'))


W1117 21:17:40.230575 13916 deprecation_wrapper.py:119] From C:\Users\VKE\AppData\Local\Continuum\anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:66: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

C:\Users\VKE\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), input_shape=(28, 28, 1...)`
  This is separate from the ipykernel package so we can avoid doing imports until
W1117 21:17:40.420890 13916 deprecation_wrapper.py:119] From C:\Users\VKE\AppData\Local\Continuum\anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:541: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1117 21:17:40.429734 13916 deprecation_wrapper.py:119] From C:\Users\VKE\AppData\Local\Continuum\anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4432: The name tf.random_uniform is deprecat

### Print the training and test accuracy for 5 epochs

In [14]:
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
model.fit(x_train_lt5, y_train_lt5,batch_size=batch_size, nb_epoch=nb_epoch,verbose=1,validation_data=(x_test_lt5, y_test_lt5))
score = model.evaluate(x_test_lt5, y_test_lt5, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

W1117 21:18:04.435805 13916 deprecation_wrapper.py:119] From C:\Users\VKE\AppData\Local\Continuum\anaconda3\lib\site-packages\keras\optimizers.py:793: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W1117 21:18:04.458337 13916 deprecation_wrapper.py:119] From C:\Users\VKE\AppData\Local\Continuum\anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:3576: The name tf.log is deprecated. Please use tf.math.log instead.

C:\Users\VKE\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  
W1117 21:18:04.653177 13916 deprecation.py:323] From C:\Users\VKE\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same 

Train on 30596 samples, validate on 5139 samples
Epoch 1/5
30596/30596 [==============================] - 40s 1ms/step - loss: 0.5991 - acc: 0.7866 - val_loss: 0.1095 - val_acc: 0.9677
Epoch 2/5
30596/30596 [==============================] - 38s 1ms/step - loss: 0.2186 - acc: 0.9324 - val_loss: 0.0796 - val_acc: 0.9749
Epoch 3/5
30596/30596 [==============================] - 38s 1ms/step - loss: 0.1796 - acc: 0.9450 - val_loss: 0.0684 - val_acc: 0.9780
Epoch 4/5
30596/30596 [==============================] - 37s 1ms/step - loss: 0.1623 - acc: 0.9480 - val_loss: 0.0600 - val_acc: 0.9805
Epoch 5/5
30596/30596 [==============================] - 36s 1ms/step - loss: 0.1506 - acc: 0.9556 - val_loss: 0.0541 - val_acc: 0.9815
Test score: 0.05414174501803175
Test accuracy: 0.9815139132938767


### Use the model trained on 0 to 4 digit classification and train it on the dataset which has digits 5 to 9  (Using Transfer learning keeping only the dense layers to be trainable)

### Make only the dense layers to be trainable and convolutional layers to be non-trainable

#### Check model summary to see model layer names

In [15]:
for layers in model.layers:
    print(layers.name)
    if('dense' not in layers.name):
        layers.trainable = False
        print(layers.name + 'is not trainable\n')
    if('dense' in layers.name):
        print(layers.name + ' is trainable\n')

conv2d_1
conv2d_1is not trainable

activation_1
activation_1is not trainable

conv2d_2
conv2d_2is not trainable

activation_2
activation_2is not trainable

max_pooling2d_1
max_pooling2d_1is not trainable

dropout_1
dropout_1is not trainable

flatten_1
flatten_1is not trainable

dense_1
dense_1 is trainable

dropout_2
dropout_2is not trainable

dense_2
dense_2 is trainable



### Do the required preprocessing for `x_train_gt5` also same as `x_train_lt5` and for `y_train_gt5` same as `y_train_lt5`

1. Reshape
2. Change to float32 datatype
3. Normalize (dividing with 255)
4. y_train and y_test Convert into one-hot vectors

In [16]:
x_train_gt5 = x_train_gt5.reshape(x_train_gt5.shape[0],28,28,1)
x_test_gt5 = x_test_gt5.reshape(x_test_gt5.shape[0],28,28,1)

x_train_gt5 = x_train_gt5.astype('float32')
x_test_gt5 = x_test_gt5.astype('float32')
x_train_gt5 /= 255
x_test_gt5 /= 255

print('X_train shape:', x_train_gt5.shape)
print(x_train_gt5.shape[0], 'train samples')
print(x_test_gt5.shape[0], 'test samples')

print('X_train shape:', x_train_gt5.shape)
print('X_test shape:', x_test_gt5.shape)

# convert class vectors to binary class matrices
y_train_gt5 = np_utils.to_categorical(y_train_gt5, nb_classes)
y_test_gt5 = np_utils.to_categorical(y_test_gt5, nb_classes)

X_train shape: (29404, 28, 28, 1)
29404 train samples
4861 test samples
X_train shape: (29404, 28, 28, 1)
X_test shape: (4861, 28, 28, 1)


### Check

Verify the shapes with the given below.

In [17]:
print(x_train_gt5.shape)
print(y_train_gt5.shape)
print(x_test_gt5.shape)
print(y_test_gt5.shape)

(29404, 28, 28, 1)
(29404, 5)
(4861, 28, 28, 1)
(4861, 5)


## Print the accuracy for classification of digits 5 to 9

In [18]:
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
model.fit(x_train_gt5, y_train_gt5,batch_size=batch_size, nb_epoch=nb_epoch,verbose=1,validation_data=(x_test_gt5, y_test_gt5))
score = model.evaluate(x_test_gt5, y_test_gt5, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

C:\Users\VKE\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  


Train on 29404 samples, validate on 4861 samples
Epoch 1/5
29404/29404 [==============================] - 18s 607us/step - loss: 0.6831 - acc: 0.7555 - val_loss: 0.2398 - val_acc: 0.9266
Epoch 2/5
29404/29404 [==============================] - 16s 558us/step - loss: 0.4001 - acc: 0.8643 - val_loss: 0.1742 - val_acc: 0.9461
Epoch 3/5
29404/29404 [==============================] - 15s 519us/step - loss: 0.3384 - acc: 0.8878 - val_loss: 0.1521 - val_acc: 0.9531
Epoch 4/5
29404/29404 [==============================] - 16s 559us/step - loss: 0.2999 - acc: 0.9026 - val_loss: 0.1385 - val_acc: 0.9582
Epoch 5/5
29404/29404 [==============================] - 17s 564us/step - loss: 0.2838 - acc: 0.9054 - val_loss: 0.1391 - val_acc: 0.9597
Test score: 0.13905038368711745
Test accuracy: 0.9596790785138143


# Text classification using TF-IDF

###  Load the dataset from sklearn.datasets

In [19]:
from sklearn.datasets import fetch_20newsgroups

In [20]:
categories = ['alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med']

In [21]:
fetch_20newsgroups

<function sklearn.datasets.twenty_newsgroups.fetch_20newsgroups(data_home=None, subset='train', categories=None, shuffle=True, random_state=42, remove=(), download_if_missing=True)>

### Training data

In [22]:
twenty_train = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42)

### Test data

In [23]:
twenty_test = fetch_20newsgroups(subset='test', categories=categories, shuffle=True, random_state=42)

###  a.  You can access the values for the target variable using .target attribute 
###  b. You can access the name of the class in the target variable with .target_names


In [24]:
twenty_train.target

array([1, 1, 3, ..., 2, 2, 2], dtype=int64)

In [25]:
twenty_train.target_names

['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']

In [26]:
twenty_train.data[0:5]

['From: sd345@city.ac.uk (Michael Collier)\nSubject: Converting images to HP LaserJet III?\nNntp-Posting-Host: hampton\nOrganization: The City University\nLines: 14\n\nDoes anyone know of a good way (standard PC application/PD utility) to\nconvert tif/img/tga files into LaserJet III format.  We would also like to\ndo the same, converting to HPGL (HP plotter) files.\n\nPlease email any response.\n\nIs this the correct group?\n\nThanks in advance.  Michael.\n-- \nMichael Collier (Programmer)                 The Computer Unit,\nEmail: M.P.Collier@uk.ac.city                The City University,\nTel: 071 477-8000 x3769                      London,\nFax: 071 477-8565                            EC1V 0HB.\n',
 "From: ani@ms.uky.edu (Aniruddha B. Deglurkar)\nSubject: help: Splitting a trimming region along a mesh \nOrganization: University Of Kentucky, Dept. of Math Sciences\nLines: 28\n\n\n\n\tHi,\n\n\tI have a problem, I hope some of the 'gurus' can help me solve.\n\n\tBackground of the probl

### Now with dependent and independent data available for both train and test datasets, using TfidfVectorizer fit and transform the training data and test data and get the tfidf features for both

In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(min_df=0, max_df=1.0, ngram_range=(1,1))
train_tfidf_features = vectorizer.fit_transform(twenty_train.data).astype(float)

In [28]:
vectorizer

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=0, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [29]:
vectorizer.vocabulary_

{'from': 14887,
 'sd345': 29022,
 'city': 8696,
 'ac': 4017,
 'uk': 33256,
 'michael': 21661,
 'collier': 9031,
 'subject': 31077,
 'converting': 9805,
 'images': 17366,
 'to': 32493,
 'hp': 16916,
 'laserjet': 19780,
 'iii': 17302,
 'nntp': 23122,
 'posting': 25663,
 'host': 16881,
 'hampton': 16082,
 'organization': 23915,
 'the': 32142,
 'university': 33597,
 'lines': 20253,
 '14': 587,
 'does': 12051,
 'anyone': 5201,
 'know': 19458,
 'of': 23610,
 'good': 15576,
 'way': 34755,
 'standard': 30623,
 'pc': 24651,
 'application': 5285,
 'pd': 24677,
 'utility': 33915,
 'convert': 9801,
 'tif': 32391,
 'img': 17389,
 'tga': 32116,
 'files': 14281,
 'into': 18268,
 'format': 14676,
 'we': 34775,
 'would': 35312,
 'also': 4808,
 'like': 20198,
 'do': 12014,
 'same': 28619,
 'hpgl': 16927,
 'plotter': 25361,
 'please': 25337,
 'email': 12833,
 'any': 5195,
 'response': 27836,
 'is': 18474,
 'this': 32270,
 'correct': 9932,
 'group': 15837,
 'thanks': 32135,
 'in': 17556,
 'advance': 4378,

In [30]:
vectorizer.get_feature_names

<bound method CountVectorizer.get_feature_names of TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=0, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)>

In [31]:
test_tfidf_features = vectorizer.transform(twenty_test.data).astype(float)

### Use logisticRegression with tfidf features as input and targets as output and train the model and report the train and test accuracy score

In [32]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [33]:
# use logistic regression with text column only
logreg = LogisticRegression()
logreg.fit(train_tfidf_features, twenty_train.target)
y_pred_class = logreg.predict(test_tfidf_features)
print(metrics.accuracy_score(y_pred_class, twenty_test.target))

C:\Users\VKE\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\VKE\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.8868175765645806


In [34]:
y_train_class = logreg.predict(train_tfidf_features)

In [35]:
print(metrics.accuracy_score(y_train_class, twenty_train.target))

0.9827204253433761
